In [1]:
# RUN AUTOENCODER JUPYTER NOTEBOOK FIRST, this will create the `encoder.h5` file used below

import keras
from keras.models import load_model

from keras.models import Model

print("Loading models...")

res_net = keras.applications.resnet50.ResNet50(include_top=False, pooling='avg')

encoder = load_model('model/encoder.h5')

print("Combining models...")
combined = Model(inputs=res_net.input, outputs=encoder(res_net.output))


Using TensorFlow backend.


Loading models...
Instructions for updating:
Colocations handled automatically by placer.
Combining models...


/usr/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [2]:
import db.mariadb as mariadb
import numpy as np

from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from PIL.Image import DecompressionBombError
# Change according to your database setup (these are the defaults, see .env file)

con = mariadb.get_connection("127.0.0.1", 3308, "image_processing", "user", "user_pw")

files_batch = mariadb.get_all_files_data(con)


In [3]:
features = []
features_info = []

# Create training and test features
print("Creating image features using both models.")
counter = 1

for (img_id, name, path, url) in files_batch:
    try:
        if(counter % 100 == 0):
            print(f'Progress: {counter}/{len(files_batch)}')
        
        img = image.load_img(path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        
        combined_feature = combined.predict(img_data)
        combined_feature = np.array(combined_feature).flatten()
        
        res_net_feature = res_net.predict(img_data)
        res_net_feature = np.array(res_net_feature).flatten()
        
        mariadb.write_file_features(img_id, combined_feature, con)
        mariadb.write_uncompressed_file_features(img_id, res_net_feature, con)

        counter += 1

    except OSError as e:
        print(e)
    except DecompressionBombError as e:
        print(e)
        print(path)
        
con.close()
print('Done.')


Creating image features using combined model.
Progress: 100/15000
Progress: 200/15000
Progress: 300/15000
Progress: 400/15000
Progress: 500/15000
Progress: 600/15000
Progress: 700/15000
Progress: 800/15000
Progress: 900/15000
Progress: 1000/15000
Progress: 1100/15000
Progress: 1200/15000
Progress: 1300/15000
Progress: 1400/15000
Progress: 1500/15000
Progress: 1600/15000
Progress: 1700/15000
Progress: 1800/15000
Progress: 1900/15000
Progress: 2000/15000
Progress: 2100/15000
Progress: 2200/15000
Progress: 2300/15000
Progress: 2400/15000
Progress: 2500/15000
Image size (207616860 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.
/mnt/idai_cloud/dai-rom-fotothek-2007/Bestand-D-DAI-ROM-2007.5572.JPG
Progress: 2600/15000
Progress: 2700/15000
Progress: 2800/15000
Progress: 2900/15000
Progress: 3000/15000
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0636.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothe

/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (124167394 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Progress: 4700/15000
Progress: 4800/15000
Progress: 4900/15000
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0577/D-DAI-ROM-71.623R_14505.jpg'


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (105211691 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Progress: 5000/15000
Progress: 5100/15000
Progress: 5200/15000
Progress: 5300/15000
Progress: 5400/15000


...

Progress: 7400/15000
Progress: 7500/15000
Progress: 7600/15000
Progress: 7700/15000


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (97257268 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Progress: 7800/15000
Progress: 7900/15000
Progress: 8000/15000
Progress: 8100/15000
Progress: 8200/15000
Progress: 8300/15000
Progress: 8400/15000
Progress: 8500/15000
Progress: 8600/15000
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0270.JPG'
Progress: 8700/15000
Progress: 8800/15000
Progress: 8900/15000
Progress: 9000/15000
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0672.JPG'
Progress: 9100/15000
Progress: 9200/15000
Progress: 9300/15000


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (144643345 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Progress: 9400/15000
Progress: 9500/15000
Progress: 9600/15000
Progress: 9700/15000
Progress: 9800/15000
Progress: 9900/15000
Progress: 10000/15000
Progress: 10100/15000
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0188.JPG'
Progress: 10200/15000
Progress: 10300/15000
Progress: 10400/15000
Progress: 10500/15000
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0095.JPG'
Progress: 10600/15000
Progress: 10700/15000
Progress: 10800/15000
Progress: 10900/15000
Progress: 11000/15000
Progress: 11100/15000
Progress: 11200/15000
Progress: 11300/15000
Progress: 11400/15000
Progress: 11500/15000
Progress: 11600/15000
Progress: 11700/15000
Progress: 11800/15000
Progress: 11900/15000
Progress: 12000/15000
Progress: 12100/15000
Progress: 12200/15000
Progress: 12300/15000
Progress: 12400/15000
Progress: 12500/15000
Progress: 12600/15000
Progress: 12700/15000
cannot identify image file '/mnt/idai_cloud/FADatenbankabb07